# Create a Dynamic STAC backend

By default cogeo-mosaic backends were meant to handle writing and reading mosaicjson either from a file or from a database.

While this is fine for most use cases, some users could want something more `dynamic`. In this Notebook we will show how to create a Dynamic mosaic backend based on STAC api.



## Requirements

To be able to run this notebook you'll need the following requirements:
- cogeo-mosaic

In [ ]:
# Uncomment this line if you need to install the dependencies
# !pip install cogeo-mosaic

In [ ]:
from typing import Dict, Tuple, Type, Optional, List

import attr
from morecantile import TileMatrixSet
from rio_tiler.constants import WEB_MERCATOR_TMS
from rio_tiler.io import BaseReader
from rio_tiler.io import STACReader

from cogeo_mosaic.backends.base import BaseBackend
from cogeo_mosaic.backends.stac import _fetch, default_stac_accessor
from cogeo_mosaic.mosaic import MosaicJSON


@attr.s
class DynamicStacBackend(BaseBackend):
    """Like a STAC backend but dynamic"""

    path: str = attr.ib()
    reader: Type[BaseReader] = attr.ib(default=STACReader)
    reader_options: Dict = attr.ib(factory=dict)
    backend_options: Dict = attr.ib(factory=dict)

    query: Dict = attr.ib(factory=dict)

    # default values for bounds and zoom
    bounds: Tuple[float, float, float, float] = attr.ib(
        default=(-180, -90, 180, 90)
    )
    minzoom: int = attr.ib(default=0)
    maxzoom: int = attr.ib(default=30)

    # Because we are not using mosaicjson we are not limited to the WebMercator TMS
    tms: TileMatrixSet = attr.ib(default=WEB_MERCATOR_TMS)

    # The reader is read-only, we can't pass mosaic_def to the init method
    mosaic_def: MosaicJSON = attr.ib(init=False)

    _backend_name = "DynamicSTAC"

    def __attrs_post_init__(self):
        """Post Init."""
        # Construct a FAKE mosaicJSON
        # mosaic_def has to be defined. As we do for the DynamoDB and SQLite backend
        # we set `tiles` to an empty list.
        self.mosaic_def = MosaicJSON(
            mosaicjson="0.0.2",
            name="it's fake but it's ok",
            minzoom=self.minzoom,
            maxzoom=self.maxzoom,
            tiles=[]
        )

    def write(self, overwrite: bool = True):
        """This method is not used but is required by the abstract class."""
        pass

    def update(self):
        """We overwrite the default method."""
        pass

    def _read(self) -> MosaicJSON:
        """This method is not used but is required by the abstract class."""
        pass

    def assets_for_tile(self, x: int, y: int, z: int) -> List[str]:
        """Retrieve assets for tile."""
        bounds = self.tms.bounds(x, y, z)
        geom = {
            "type": "Polygon",
            "coordinates": [
                [
                    [bounds[0], bounds[3]],
                    [bounds[0], bounds[1]],
                    [bounds[2], bounds[1]],
                    [bounds[2], bounds[3]],
                    [bounds[0], bounds[3]],
                ]
            ],
        }
        return self.get_assets(geom)

    def assets_for_point(self, lng: float, lat: float) -> List[str]:
        """Retrieve assets for point."""
        EPSILON = 1e-14
        geom = {
            "type": "Polygon",
            "coordinates": [
                [
                    [lng - EPSILON, lat + EPSILON],
                    [lng - EPSILON, lat - EPSILON],
                    [lng + EPSILON, lat - EPSILON],
                    [lng + EPSILON, lat + EPSILON],
                    [lng - EPSILON, lat + EPSILON],
                ]
            ],
        }
        return self.get_assets(geom)

    def get_assets(self, geom) -> List[str]:
        """Find assets."""
        query = self.query.copy()
        query["intersects"] = geom

        features = _fetch(
            self.path,
            query,
            **self.backend_options,
        )
        return [default_stac_accessor(f) for f in features]

    @property
    def _quadkeys(self) -> List[str]:
        return []


In [ ]:
##  Base Query for sat-api
# - limit to 5 assets
# - less than 25% of cloud
# - more than 75% of data coverage
# - `sentinel-s2-l2a-cogs` collection
query = {
    "collections": ["sentinel-s2-l2a-cogs"],
    "limit": 5,
    "query": {
        "eo:cloud_cover": {
            "lt": 25
        },
        "sentinel:data_coverage": {
            "gt": 75
        }
    },
    "fields": {
      'include': ['id'],
      'exclude': ['assets', 'geometry']
    }
}

In [ ]:
# Read Tile
with DynamicStacBackend(
    "https://earth-search.aws.element84.com/v0/search",
    query=query,
    backend_options={"max_items":5},
) as mosaic:
    img, _ = mosaic.tile(535, 335, 10, assets="B01", tilesize=128, threads=0)

print(img)

In [ ]:
# Read Point values
with DynamicStacBackend(
    "https://earth-search.aws.element84.com/v0/search",
    query=query,
    backend_options={"max_items":5},
) as mosaic:
    values = mosaic.point(-1.0546875,  51.99, assets="B01")

In [ ]:
print(values)